In [ ]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.output_parsers import StrOutputParser

In [ ]:
embeddings_model = OpenAIEmbeddings()

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    max_tokens=300,
)

In [ ]:
pdf_path = "os-sertoes.pdf"
loader = PyPDFLoader(pdf_path, extract_images=False)
pages = loader.load_and_split()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=30,
    length_function=len,
    add_start_index=True,
)

chunks = text_splitter.split_documents(pages)
db = Chroma.from_documents(chunks, embedding=embeddings_model)
retriever = db.as_retriever(search_kwargs={"k": 3})

In [ ]:
TEMPLATE = """
Você é um especialista no livro Os Sertões. Responda segundo o contexto.

Contexto: {context}
    
Pergunta: {question}
"""

prompt = PromptTemplate(input_variables=["context", "question"], template=TEMPLATE)
parser = StrOutputParser()
sequence = RunnableSequence(prompt | llm | parser)

In [ ]:
def answer(question):
    context = retriever.invoke(question)
    response = sequence.invoke({"context": context, "question": question})
    return response

In [ ]:
questions = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em 'Os Sertões'? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",
]

for question in questions:
    response = answer(question)
    print({"pergunta": question, "resposta": response})